# Fully-Connected Optimization

In [ ]:
import os
from os.path import join
import sys
from pathlib import Path

# include app directory into sys.path
parent_dir = Path(os.path.abspath('')).parent
app_dir = join(parent_dir, "app")
if app_dir not in sys.path:
      sys.path.append(app_dir)

import torch as pt
import matplotlib.pyplot as plt
import numpy as np
from utils import config

# increase plot resolution
plt.rcParams["figure.dpi"] = 180

# retrieve parameters from config
DIM_REDUCTION = "SVD"       # one of ("SVD" / "VAE")
PRED_HORIZON = config.FC_SVD_pred_horizon if DIM_REDUCTION == "SVD" else config.FC_VAE_pred_horizon
EPOCHS = config.FC_SVD_single_epochs if DIM_REDUCTION == "SVD" else config.FC_VAE_single_epochs

# define paths
FC_PATH = join(parent_dir, "output", "FC", "single", DIM_REDUCTION, "param_study", f"pred_horizon_{PRED_HORIZON}")
OUTPUT_PATH = join(parent_dir, "output", "FC", "single", DIM_REDUCTION, "param_study")

#### Evaluate study results

In [ ]:
# load study results
study_results = pt.load(join(FC_PATH, "study_results.pt"))
param_combinations = list(study_results.keys())

# find parameter combinations of study and extract test loss
hidden_size = np.unique([int(param_set.split('_')[1]) for param_set in param_combinations])
n_hidden = np.unique([int(param_set.split('_')[2]) for param_set in param_combinations])

In [ ]:
# Calculate the mean and standard deviation of test loss for each epoch for each parameter combination
train_means = {}
test_means = {}
best_models = {}

for param_combination in study_results:
    all_test_losses = []
    all_train_losses = []
    for df in study_results[param_combination]:
        all_test_losses.append(df["val_loss"])
        all_train_losses.append(df["train_loss"])

    # identify model with lowest test loss for each parameter combination
    final_losses = [test_loss[-10:].mean() for test_loss in all_test_losses]
    best_models[str(final_losses.index(min(final_losses)) + 1) + "_" + param_combination] = min(final_losses)
    
    # Pad shorter training sequences with NaNs to ensure equal lengths
    all_test_losses_padded = np.array([np.pad(loss, (0, EPOCHS - len(loss)), mode='constant', constant_values=np.nan) for loss in all_test_losses])
    all_train_losses_padded = np.array([np.pad(loss,(0, EPOCHS - len(loss)), mode='constant', constant_values=np.nan) for loss in all_train_losses])
    
    # Calculate mean and std while ignoring NaN values
    test_means[param_combination] = np.nanmean(all_test_losses_padded, axis=0)
    train_means[param_combination] = np.nanmean(all_train_losses_padded, axis=0)

# Create the plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 3), sharey = True, sharex = True)

# Plot mean training and test loss
for size in study_results:
    ax1.plot(np.arange(1, EPOCHS + 1), train_means[size], label=f'{size.split("_")[1]} Neur., {size.split("_")[2]} Layer')
    ax2.plot(np.arange(1, EPOCHS + 1), test_means[size], label=f'{size.split("_")[1]} Neur., {size.split("_")[2]} Layer')

ax1.set_xlabel("epoch")
ax1.set_yscale("log")
ax1.set_title("Train MSE Mean")
ax2.set_xlabel("epoch")
ax2.set_yscale("log")
ax2.set_title("Test MSE Mean")
#ax2.legend(loc=1, bbox_to_anchor=(1,1))
ax1.grid()
ax2.grid()
plt.xlim(0, EPOCHS)
plt.tight_layout()
plt.savefig(join(OUTPUT_PATH, f"{DIM_REDUCTION}_FC_single_predhor{PRED_HORIZON}_loss_mean.png"), bbox_inches="tight")